In [46]:
#Course 3 goal: now instead of using sigmoid, use voltage
import math
import numpy as np
import random
restingpotential= -65
threshold= -15 

# Define the Neuron class
class Neuron:
    def __init__(self): #setting the beginning
        self.activation = 0
        self.voltage = restingpotential
        self.threshold= threshold
        #activation should be zero in the beginning

    def signal_integration(self, vsignals, sweight):
        self.vsignals = vsignals
        self.sweight = sweight
        self.ssum = sum(self.vsignals * self.sweight)
        self.voltage = self.voltage + self.ssum - restingpotential

        # Add a random noise term
        noise = random.uniform(-10, 10)  # Adjust the range of noise as needed
        self.voltage += noise

        # Adjust the threshold value to control the firing rate
        threshold_adjusted = self.threshold + random.uniform(-10, 10)  # Adjust the range of threshold adjustment as needed
        self.activation = 1 if self.voltage >= threshold_adjusted else 0



        
# Define the Layer class
class Layer:
    def __init__(self, layer_index, n_neurons):
        self.layer_index = layer_index
        self.n_neurons = n_neurons
        self.neuron_layer = [Neuron() for _ in range(n_neurons)]


# Define the Connection class
class Connection:
    def __init__(self, previous_layer_n, next_layer_n):
        self.previous_layer_n = previous_layer_n
        self.next_layer_n = next_layer_n
        self.matrix_connections = np.zeros((self.previous_layer_n, self.next_layer_n))

        # Connection matrix: the number of rows is the previous layer, and the columns represent the next layer
        for i in range(self.previous_layer_n):  # Number of rows
            for j in range(self.next_layer_n):  # Number of columns
                self.matrix_connections[i][j] = random.random()


# Define the Network class, same as previous class but now the number of neurons and layers are set
class Network:
    def __init__(self):
        self.n_layers = 50  # Number of layers
        self.network = []

        # Create the layers
        for i in range(self.n_layers):
            n_neurons = 50  # Number of neurons in each layer
            layer = Layer(i, n_neurons)
            self.network.append(layer)

        # Create the matrices for the connections
        self.connections = []

        for i in range(self.n_layers - 1):
            n_previous_layer = len(self.network[i].neuron_layer)  # Number of neurons in the previous layer
            n_next_layer = len(self.network[i + 1].neuron_layer)  # Number of neurons in the next layer
            connection = Connection(n_previous_layer, n_next_layer)
            self.connections.append(connection.matrix_connections)

    def signal_stream(self, input_vec):
        self.inputs = input_vec

    # Compute the first layer update
        for i in range(len(self.network[0].neuron_layer)):
            if i < len(self.inputs):
                self.network[0].neuron_layer[i].activation = self.inputs[i]
            else:
                self.network[0].neuron_layer[i].activation = 0.0

    # Loop over the layers (from the 2nd to the last layer)
        for i in range(self.n_layers - 1):
            n_neurons = len(self.network[i + 1].neuron_layer)
            self.new_inputs = np.zeros(n_neurons)
            weight_matrix = self.connections[i]

            for a in range(n_neurons):
                weight = weight_matrix[:, a]
                self.network[i + 1].neuron_layer[a].signal_integration(self.inputs, weight)
                self.new_inputs[a] = self.network[i + 1].neuron_layer[a].activation

            self.inputs = self.new_inputs.copy()  # Update with the activation values of the current layer



    def system_state(self, threshold=0):
        for i in range(self.n_layers):
            state_vec = np.zeros(len(self.network[i].neuron_layer))
            voltage_vec = np.zeros(len(self.network[i].neuron_layer))
            fired_vec = np.zeros(len(self.network[i].neuron_layer), dtype=bool)

            for a in range(len(self.network[i].neuron_layer)):
                state_vec[a] = self.network[i].neuron_layer[a].activation
                voltage = self.network[i].neuron_layer[a].activation - restingpotential
                voltage_vec[a] = voltage
                fired_vec[a] = voltage >= threshold

            print(f"Layer {i + 1} has activation values:", state_vec)
            print(f"Layer {i + 1} has voltage values:", voltage_vec)
            print(f"Layer {i + 1} neuron firing status:", fired_vec)


#now determine which is the "winner"
    def winner(self):
        counter = 0
        highest_val = self.network[-1].neuron_layer[0].activation

        for i in range(self.network[-1].n_neurons):
            new_value = self.network[-1].neuron_layer[i].activation
            
            if new_value > highest_val:
                highest_val = new_value
                counter = i
                
        
        print(f"For the last layer the neuron with the highest value is neuron number {counter + 1} ")
        
        
        # now run it
network = Network()

# chose the input vector, can be changed by just changing within the ()
input_vec = np.random.rand(50)

# Signal stream through the network
network.signal_stream(input_vec)

# Print
network.system_state(threshold)

# which neuron wins ?
network.winner()

Layer 1 has activation values: [0.51803248 0.92092338 0.64031928 0.45256004 0.1209605  0.06846432
 0.25752187 0.85591628 0.63314593 0.8421766  0.64803908 0.40974978
 0.11518916 0.59872391 0.02049874 0.00791271 0.88493885 0.65085589
 0.93869407 0.15150726 0.08474956 0.75951226 0.88436485 0.381834
 0.90447214 0.79851949 0.7126068  0.74602396 0.65900215 0.95706805
 0.16382009 0.72541748 0.22559021 0.35476963 0.34136143 0.55376011
 0.63295762 0.43894606 0.51234304 0.88514972 0.56998022 0.32198591
 0.20093034 0.12655227 0.24492798 0.77201299 0.79273305 0.42476593
 0.08545216 0.28082241]
Layer 1 has voltage values: [65.51803248 65.92092338 65.64031928 65.45256004 65.1209605  65.06846432
 65.25752187 65.85591628 65.63314593 65.8421766  65.64803908 65.40974978
 65.11518916 65.59872391 65.02049874 65.00791271 65.88493885 65.65085589
 65.93869407 65.15150726 65.08474956 65.75951226 65.88436485 65.381834
 65.90447214 65.79851949 65.7126068  65.74602396 65.65900215 65.95706805
 65.16382009 65.7254